# CS137 Final Project - HuBMAP - Hacking the Human Body

## Imports and environment setup

In [1]:
# If use google colab, mount the working directory there. 
from google.colab import drive
import sys
drive.mount('/content/drive')

# NOTE: you need to use your own path to add the implementation to the python path 
# so you can import functions from implementation.py
sys.path.append('/content/drive/MyDrive/CS137_Final_Project')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# A bit of setup
import numpy as np
import torch
import matplotlib.pyplot as plt
import sys

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Importing UNet Model functions

In [3]:
from unet_utils import *
from SimpleUNet import *

## Instantiating UNet and checking model summary

In [4]:
from torchsummary import summary
model = UNetModel()
model.to(device)
summary(model, input_size=(1, 512, 512))

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 512, 512]             640
       BatchNorm2d-2         [-1, 64, 512, 512]             128
              ReLU-3         [-1, 64, 512, 512]               0
            Conv2d-4         [-1, 64, 512, 512]          36,928
       BatchNorm2d-5         [-1, 64, 512, 512]             128
              ReLU-6         [-1, 64, 512, 512]               0
         ConvBlock-7         [-1, 64, 512, 512]               0
         MaxPool2d-8         [-1, 64, 256, 256]               0
            Conv2d-9        [-1, 128, 256, 256]          73,856
      BatchNorm2d-10        [-1, 128, 256, 256]             256
             ReLU-11        [-1, 128, 256, 256]               0
           Conv2d-12        [-1, 128, 256, 256]         147,584
      BatchNorm2d-13        [-1, 128, 256, 256]             256
             ReLU-14        [-1, 128, 2

## Training the Model

In [ ]:
import numpy as np

from loss_functions import *
from torchvision.transforms import ToTensor, Compose
from torch.utils.data import random_split, DataLoader


def train():
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(device)

    # from bean_dataset import BeanImageDataset
    #
    # trainset = BeanImageDataset("/content/drive/MyDrive/CS137_Assignment3_RobPitkin/data/train")
    # validset = BeanImageDataset("/content/drive/MyDrive/CS137_Assignment3_RobPitkin/data/validation")

    train_loader = DataLoader(trainset, batch_size=64, shuffle=True)
    valid_loader = DataLoader(validset, batch_size=64, shuffle=True)

    if device.type == "cuda":
        total_mem = torch.cuda.get_device_properties(0).total_memory
    else:
        total_mem = 0

    epochs = 100
    learning_rate = 0.00001

    model = UNetModel()
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = DiceLoss(mode='multiclass', from_logits=True)

    # Recording the loss
    train_loss = []
    val_loss = []
    val_acc = []

    for i in range(epochs):
        running_train_loss = 0.0
        for j, data in enumerate(train_loader):
            x, y = data
            ## If GPU is available, move to cuda
            if device.type == "cuda":
                x = x.to(device)
                y = y.to(device)

            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()

            if device.type == "cuda":
                loss = loss.cpu()

            running_train_loss += np.mean(loss.data.numpy())

        running_train_loss /= train_loader.__len__()
        train_loss.append(running_train_loss)

        # validate
        running_val_loss = 0.0
        running_val_acc = 0.0
        with torch.no_grad():
            for k, data in enumerate(valid_loader):
                x, y = data
                if device.type == "cuda":
                    x = x.to(device)
                    y = y.to(device)
                output = model(x)
                loss = criterion(output, y)

                if device.type == "cuda":
                    loss = loss.cpu()

                running_val_loss += np.mean(loss.data.numpy())

                for l in range(len(x)):
                    pred = torch.nn.functional.softmax(output[l], dim=0)
                    if torch.argmax(pred) == y[l]:
                        running_val_acc += 1

            running_val_acc /= len(valid_loader.dataset.y)
            running_val_loss /= valid_loader.__len__()
            if (len(val_loss) != 0 and np.mean(running_val_loss) < min(val_loss)):
                torch.save(model.state_dict(), 'best.sav')
            val_loss.append(running_val_loss)
            val_acc.append(running_val_acc)

        # check GPU memory if necessary
        if device.type == "cuda":
            alloc_mem = torch.cuda.memory_allocated(0)
        else:
            alloc_mem = 0

        # print out
        print(
            f"Epoch [{i + 1}]: Training Loss: {running_train_loss} Validation Loss: {running_val_loss} Accuracy: {running_val_acc}" + (
                f" Allocated/Total GPU memory: {alloc_mem}/{total_mem}" if device.type == "cuda" else ""
            ))


## Plot loss and accuracy curves

In [ ]:
plt.subplot(2, 1, 1)
plt.plot(train_loss, '-o')
plt.plot(val_loss, '-o')
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('iteration')
plt.ylabel('loss')

plt.subplot(2, 1, 2)
plt.plot(val_acc, '-o')

plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

## Save the current model (optional)

In [ ]:
torch.save(model, "UNet.sav")

## Testing the Model

In [ ]:
# Load the previously saved model.
# testset = BeanImageDataset("data/test")
# test_loader = DataLoader(testset, batch_size=1, shuffle=True)

# Load the saved model
# model = torch.load("UNet.sav")

# Load the best model
model.load_state_dict(torch.load('best'))
model.eval()


t_acc = []
with torch.no_grad():
    # only one item in the iterator
    # Add more batches if your device couldn't handle the computation 
    for _, data in enumerate(test_loader):
        x, y = data
        x = x.to(device)
        y = y.to(device)
        y_hat = model(x)

        if device.type == "cuda":
          x = x.to("cpu")
          y = y.to("cpu")
          y_hat = y_hat.to("cpu")
        acc = np.average(y.numpy() == np.argmax(y_hat.numpy(), axis = 1))
        t_acc.append(acc.item())
print(f"Test accuracy: {np.average(t_acc)}")